In [2]:
import datetime as dt
import pandas as pd
import joblib

In [3]:
df_train = pd.read_csv('training.csv', index_col='Date') # 2003 - 2013
df_val = pd.read_csv('validation.csv', index_col='Date') # 2017/08/3
df_pred = pd.read_csv('prediction.csv', index_col='Date') # 2024/07/23

MODEL_PATH = "model.pkl"
model = joblib.load(MODEL_PATH)

In [4]:
# no need to run this now, this was code to setup the csv file
display(df_train.columns)

#display(df_val)
df_pred.drop(columns=['Quarterly Revenue Growth  (yoy)', 'Operating Cash Flow  (ttm)'], inplace=True)

df_pred.rename(columns={
    'PEG Ratio (5yr expected)': 'PEG Ratio',
    'Operating Margin  (ttm)': 'Operating Margin',
    'Return on Assets  (ttm)': 'Return on Assets',
    'Return on Equity  (ttm)': 'Return on Equity', 
    'Revenue  (ttm)': 'Revenue',
    'Revenue Per Share  (ttm)': 'Revenue Per Share',
    'Net Income Avi to Common  (ttm)': 'Net Income Avl to Common',
    'Diluted EPS  (ttm)': 'Diluted EPS',
    'Total Cash  (mrq)': 'Total Cash',
    'Total Cash Per Share  (mrq)': 'Total Cash Per Share',
    'Total Debt  (mrq)': 'Total Debt',
    'Total Debt/Equity  (mrq)': 'Total Debt/Equity',
    'Current Ratio  (mrq)': 'Current Ratio',
    'Book Value Per Share  (mrq)': 'Book Value Per Share',
    'Beta (5Y Monthly)': 'Beta',
    '50-Day Moving Average3': '50-Day Moving Average',
    '200-Day Moving Average3': '200-Day Moving Average',
    'Avg Vol (3 month)3': 'Avg Vol (3 month)',
    'Shares Outstanding5': 'Shares Outstanding'

}, inplace=True)

display(df_pred.columns)

df1_columns = [col for i, col in enumerate(df_train.columns) if i not in [1, 2, 3, 4]]
df2_columns = [col for i, col in enumerate(df_pred.columns) if i not in []]

for col1, col2 in zip(df1_columns, df2_columns):
    print(f"df1: {col1} df2: {col2}")

Index(['Ticker', 'Price', 'stock_p_change', 'SP500', 'SP500_p_change',
       'Market Cap', 'Enterprise Value', 'Trailing P/E', 'Forward P/E',
       'PEG Ratio', 'Price/Sales', 'Price/Book', 'Enterprise Value/Revenue',
       'Enterprise Value/EBITDA', 'Profit Margin', 'Operating Margin',
       'Return on Assets', 'Return on Equity', 'Revenue', 'Revenue Per Share',
       'EBITDA', 'Net Income Avl to Common', 'Diluted EPS', 'Total Cash',
       'Total Cash Per Share', 'Total Debt', 'Total Debt/Equity',
       'Current Ratio', 'Book Value Per Share', 'Beta',
       '50-Day Moving Average', '200-Day Moving Average', 'Avg Vol (3 month)',
       'Shares Outstanding'],
      dtype='object')

Index(['Ticker', 'Market Cap', 'Enterprise Value', 'Trailing P/E',
       'Forward P/E', 'PEG Ratio', 'Price/Sales', 'Price/Book',
       'Enterprise Value/Revenue', 'Enterprise Value/EBITDA', 'Profit Margin',
       'Operating Margin', 'Return on Assets', 'Return on Equity', 'Revenue',
       'Revenue Per Share', 'EBITDA', 'Net Income Avl to Common',
       'Diluted EPS', 'Total Cash', 'Total Cash Per Share', 'Total Debt',
       'Total Debt/Equity', 'Current Ratio', 'Book Value Per Share', 'Beta',
       '50-Day Moving Average', '200-Day Moving Average', 'Avg Vol (3 month)',
       'Shares Outstanding', 'Predictions'],
      dtype='object')

In [4]:
X = df_pred.iloc[:, 1:]

In [5]:
predictions = model.predict(X)

In [6]:
preds = pd.Series(predictions, name='Predictions')
#X_original = pd.DataFrame(scaler.inverse_transform(X), columns=X.columns)
merged = pd.concat([df_pred.reset_index(drop=True), preds.reset_index(drop=True)], axis=1)

stock_symbols = merged[merged['Predictions'] == True]['Ticker'].to_list()

In [11]:
start = dt.datetime(2024, 7, 23)
end = dt.datetime.now()
results = []
average = 0
import yfinance as yf

for stock in ['SPY']:
    df = yf.download(stock, start=start, end=end)['Adj Close']
    if len(df) > 0:
        pct_change = df[-1] / df[0]
        results.append((stock, pct_change))
        average += pct_change

for stock, pct_change in results:
    print(f"Stock: {stock}, Percentage Change: {(pct_change-1):.2%}")
print(f"{(average/len(results) - 1) * 100} %")

[*********************100%%**********************]  1 of 1 completed

Stock: SPY, Percentage Change: 1.51%
1.5078145011006017 %



C:\Users\Bogdan\AppData\Local\Temp\ipykernel_10620\3167201161.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pct_change = df[-1] / df[0]


In [14]:
merged.to_csv('final.csv')